In [15]:
import fitz
import pysubs2
from bs4 import BeautifulSoup
from sub_PDF_location import get_time_location
from PIL import Image
import cv2
import numpy as np
from moviepy.editor import VideoFileClip, AudioFileClip
from moviepy.editor import ImageClip, concatenate_videoclips
import subprocess 


In [16]:
def draw_line(pdf_file_path, page_num, top, left, length,height,color=(1,1,0), dpi=300):
    pdf_document = fitz.open(pdf_file_path)
    page=pdf_document[page_num]
    # 绘制从左下，到右下的直线
    x0=left
    y0=top+height
    x1=left+length
    y1=top+height

    shape=page.new_shape()
    shape.draw_line((x0,y0),(x1,y1)) 
    shape.finish(color=color)
    shape.commit()
    pix = page.get_pixmap(matrix=fitz.Matrix(dpi/72, dpi/72))
    return pix 


def pixmap_to_cv_image(pixmap):
    # 获取图像数据
    image_data = np.frombuffer(pixmap.samples, dtype=np.uint8)
    
    # 将图像数据重新调整为正确的形状
    if pixmap.alpha:
        shape = (pixmap.height, pixmap.width, 4)
    else:
        shape = (pixmap.height, pixmap.width, 3)
    image_data = image_data.reshape(shape)

    # 将图像从RGBA转换为BGR格式
    if pixmap.alpha:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGBA2BGR)
    else:
        image_data = cv2.cvtColor(image_data, cv2.COLOR_RGB2BGR)

    return image_data
    

In [17]:
if __name__ == '__main__':
    pdf_file_path = 'test_pdf/test2.pdf'
    vtt_file_path = 'test_pdf/test.srt'
    start_page = 1
    end_page = 1
    time_location = get_time_location(pdf_file_path, vtt_file_path, start_page, end_page,n=0)
    images=[]
    timestamps=[]
    for i, t in enumerate(time_location[:20]):
        # print(t)
        pix=draw_line(pdf_file_path, start_page+t['page_num'], t['top'], t['left'], t['line_length']/2*0.9,t['line_height'])
        pix_filename=f"test_pdf/test{i:03d}.png"
        pix.save(pix_filename)
        # cv2_img=pixmap_to_cv_image(pix)
        images.append(pix_filename)
        timestamps.append((t['start_time']/1000,t['end_time']/1000))
    


In [18]:
def generate_video(image_files, timestamps, audio_path):
    # 创建一个临时文本文件，其中列出了图片文件的路径以及每张图片的持续时间
    with open('input.txt', 'w') as f:
        for i, image_file in enumerate(image_files):
            f.write(f"file '{image_file}'\n")
            start_time, end_time = timestamps[i]
            duration = end_time - start_time
            f.write(f"duration {duration}\n")
        # 重复最后一张图片以确保其持续时间正确
        f.write(f"file '{image_files[-1]}'\n")

    # 使用ffmpeg命令读取文本文件并生成视频
    output_video_without_audio='output_video_without_audio.mp4'
    cmd = ['ffmpeg', '-f', 'concat', '-safe', '0', '-i', 'input.txt', '-vf', 'scale=ceil(iw/2)*2:ceil(ih/2)*2', '-c:v', 'h264_nvenc', '-pix_fmt', 'yuv420p', output_video_without_audio]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    
    
    # 打印捕获的输出
    print(result.stdout)
        # 将音频添加到生成的视频中
    output_video_with_audio = 'output_video_with_audio.mp4'
    command = f'ffmpeg -i {output_video_without_audio} -i {audio_path} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 {output_video_with_audio}'
    subprocess.call(command, shell=True)
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

In [19]:
generate_video(images, timestamps, 'test_pdf/test.mp3')

In [ ]:
# import imageio
# # import imageio_ffmpeg as ffmpeg
# # import numpy as np
# import subprocess 

# def create_video_with_audio(images, timestamps, audio_path, output_video, frame_rate):
#     # 获取图像尺寸
#     height, width, layers = images[0].shape

#     # 创建视频编码器
#     output_params = ['-r', str(frame_rate), '-c:v', 'libx264', '-preset', 'medium', '-profile:v', 'main', '-pix_fmt', 'yuv420p']
#     writer = imageio.get_writer(output_video, format='FFMPEG', mode='I', fps=frame_rate, codec='libx264', output_params=output_params)

#     # 将图像添加到视频中
#     for start_time, end_time in timestamps:
#         duration = end_time - start_time
#         total_frames = int(duration * frame_rate)

#         for img in images:
#             for _ in range(total_frames):
#                 writer.append_data(img)

#     # 关闭视频编码器
#     writer.close()

#     # 将音频添加到生成的视频中
#     output_video_with_audio = 'output_video_with_audio.mp4'
#     command = f'ffmpeg -i {output_video} -i {audio_path} -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 {output_video_with_audio}'
#     subprocess.call(command, shell=True)

In [ ]:
# output_video = 'output_video_no_audio.mp4' # 输出无音频的视频文件名
# final_video = 'output_video_with_audio.mp4' # 输出带音频的视频文件名
# frame_rate = 30 # 视频的帧率
# audio_path = 'test_pdf/test.mp3' # 音频文件路径

# create_video_with_audio(images, timestamps, audio_path, output_video, frame_rate)
# print("done")
